In [102]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

In [104]:
# Importing the data

df = pd.read_csv('imdb_labelled.txt', sep='  \t', header=None)
#df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
df.columns = ['Sentence', 'Score']

In [105]:
#Setting the bools for the score
df.loc[df['Score'] == 0, 'Bool'] = True
df.loc[df['Score'] == 1, 'Bool'] = False

Create a list of words that are in the negative and positive comments.

In [106]:
negative_words = pd.DataFrame([])
positive_words = pd.DataFrame([])
lower_no_punc = pd.DataFrame([])
#[^\w\d\s]+
for index in df.index:
    if df.iloc[index]['Score'] == 0:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[{}]'.format(string.punctuation), ' ', s)
        negative_words = negative_words.append(out.lower().split())

    if df.iloc[index]['Score'] == 1:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[{}]'.format(string.punctuation), ' ', s)
        positive_words = positive_words.append(out.lower().split())
    lower_no_punc = lower_no_punc.append([out.lower().strip()])

lower_no_punc = lower_no_punc.reset_index(drop=True)
lower_no_punc.columns = ['Sentence']
negative_words = pd.DataFrame(negative_words[0].unique())
positive_words = pd.DataFrame(positive_words[0].unique())

Dropping out the words that are in both positive and negative lists.

In [107]:
#Compare word lists to each other and remove matching.
neg_match = negative_words[0].isin(positive_words[0])
pos_match = positive_words[0].isin(negative_words[0])

for index in neg_match.index:
    if neg_match.iloc[index] == True:
        negative_words.drop(index, inplace=True)

for index in pos_match.index:
    if pos_match.iloc[index] == True:
        positive_words.drop(index, inplace=True)
        
negative_words = negative_words.reset_index(drop=True)
positive_words = positive_words.reset_index(drop=True)
# Drops anything that is less then three words, reducing match errors.
negative_words.drop(negative_words[negative_words[0].str.len() < 3].index, inplace=True)
positive_words.drop(positive_words[positive_words[0].str.len() < 3].index, inplace=True)

Outputting the result of the word list to see if it matches the score.

In [108]:
for neg in negative_words[0]:
    df[str(neg)] = lower_no_punc['Sentence'].str.contains(str(neg), case=False)

In [109]:
from sklearn.naive_bayes import BernoulliNB

#Instantiate the model and store in a variable.
bnb = BernoulliNB()

## Naive Bayes

In [110]:
data = df[negative_words[0]]
target = df['Bool']
target = target.astype('bool')

In [111]:
bnb.fit(data, target)

y_pred = bnb.predict(data)

print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 228


### Results

I engineered the features to separate the words from the negative and positive comments and then subtracting out words that showed up in both lists.  This helped to create a good list the is representative of the negative comments.

This method could be improved as around 27% of the comments not being accurately represented.  Leaving us with around a 70% accuracy.